# SQLite Database

This notebook is for creating an sqlite database from the exported csv files in the data\transformed_data folder. Unlike postGreSQL databases, SQLite databases do not require the use of additional software. The database is stored in a file in the database folder and can be easily accessed using sqlalchemy.

## Database Creation

In [1]:
# Load dependencies
import pandas as pd
from sqlalchemy import create_engine, MetaData, inspect, Table, Column, Integer, String, text
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [2]:
# Create sqlite database
engine = create_engine('sqlite:///us_hate_crimes_sqlite.db')

In [3]:
# Store filenames in a list
table_names = ['bias', 'census_data', 'incident', 'incident_bias',
               'incident_location', 'incident_offense', 'incident_victim_type',
               'jurisdiction', 'location', 'offense', 'race', 'state', 'victim_type']

# Loop through list, read each file into a dataframe, and then export to a table in the database
# Notes:
#  - Pandas to_sql cannot define a primary key nor can you set a primary key after creation in an sqlite database
#  - SQLalchemy can not send multiple commands to an SQLite database, hence the SQL schema file cannot be used
#  - The best method would be to define/create each table and append the corresponding dataframe
for table in table_names:
    df = pd.read_csv('../data/transformed_data/' + table + '.csv')
    df.to_sql(table, engine, if_exists='replace', index=False)
    print(f'Table created: {table}')

Table created: bias
Table created: census_data
Table created: incident
Table created: incident_bias
Table created: incident_location
Table created: incident_offense
Table created: incident_victim_type
Table created: jurisdiction
Table created: location
Table created: offense
Table created: race
Table created: state
Table created: victim_type


In [4]:
# Display a table to verify creation
state_df = pd.read_sql('select * from state', engine)
state_df.head()

,state_abbr,state,division,region
0,AK,Alaska,Pacific,West
1,AL,Alabama,East South Central,South
2,AR,Arkansas,West South Central,South
3,AZ,Arizona,Mountain,West
4,CA,California,Pacific,West


In [5]:
# Create year view in database

# Query originally developed in postgreSQL database
query = """
CREATE VIEW year_view AS
SELECT ROW_NUMBER() OVER (ORDER BY 1) AS id, i.incident_year as year
FROM incident as i
GROUP BY i.incident_year
ORDER BY i.incident_year;
"""

# Execute query to create view
with engine.connect() as con:
    con.execute(text(query))

# Read in view and display to verify creation
pd.read_sql("select * from year_view", engine)

,id,year
0,1,2009
1,2,2010
2,3,2011
3,4,2012
4,5,2013
5,6,2014
6,7,2015
7,8,2016
8,9,2017
9,10,2018


In [6]:
# Create population view in database

# Query originally developed in postgreSQL database
query = """
CREATE VIEW population_view AS
SELECT ROW_NUMBER() OVER (ORDER BY 1) AS id, c.year, s.state, sum(c.population) AS population, 'All' AS _all
FROM census_data as c
LEFT JOIN (
SELECT s.state_abbr, s.state, s.region, s.division
FROM state AS s
) s
ON s.state_abbr = c.state_abbr
GROUP BY c.year, s.state
ORDER BY c.year, s.state
"""

# Execute query to create view
with engine.connect() as con:
    con.execute(text(query))

# Read in view and display to verify creation
pd.read_sql('select * from population_view', engine)

,id,year,state,population,_all
0,1,2009,Alabama,4595018,All
1,2,2009,Alaska,671529,All
2,3,2009,Arizona,5747084,All
3,4,2009,Arkansas,2771008,All
4,5,2009,California,30669293,All
...,...,...,...,...,...
658,659,2021,Virginia,8317118,All
659,660,2021,Washington,7238408,All
660,661,2021,West Virginia,1791487,All
661,662,2021,Wisconsin,5741298,All


In [7]:
# Create incident view in database

# Query originally developed in postgreSQL database
query = """
CREATE VIEW incident_view AS
SELECT ROW_NUMBER() OVER (ORDER BY 1) AS id, i.incident_id, i.incident_year, s.state_abbr, s.state, s.region, s.division, ib.bias_category, io.offense, 'All' as _all
FROM incident AS i
LEFT JOIN (
SELECT ib.incident_id, b.bias_category AS bias_category
FROM incident_bias AS ib
LEFT JOIN (
SELECT b.bias_id, b.bias_category
FROM bias AS b
) b
ON ib.bias_id = b.bias_id
) ib
ON i.incident_id = ib.incident_id
LEFT JOIN (
SELECT io.incident_id, o.offense AS offense
FROM incident_offense AS io
LEFT JOIN (
SELECT o.offense_id, o.offense
FROM offense AS o
) o
ON io.offense_id = o.offense_id
) io
ON i.incident_id = io.incident_id
LEFT JOIN (
SELECT s.state_abbr, s.state, s.region, s.division
FROM state AS s
) s
ON i.state_abbr = s.state_abbr
ORDER BY i.incident_id
"""

# Execute query to create view
with engine.connect() as con:
    con.execute(text(query))

# Read in view and display to verify creation
pd.read_sql('select * from incident_view', engine)

,id,incident_id,incident_year,state_abbr,state,region,division,bias_category,offense,_all
0,10,136965,2009,AL,Alabama,South,East South Central,"Race, Ethnicity or Ancestry",Intimidation,All
1,11,136966,2009,AL,Alabama,South,East South Central,"Race, Ethnicity or Ancestry",Destruction/Damage/Vandalism of Property,All
2,12,136967,2009,AL,Alabama,South,East South Central,Sexual Orientation,Destruction/Damage/Vandalism of Property,All
3,13,136968,2009,AL,Alabama,South,East South Central,"Race, Ethnicity or Ancestry",Intimidation,All
4,14,136969,2009,AL,Alabama,South,East South Central,Religion,Destruction/Damage/Vandalism of Property,All
...,...,...,...,...,...,...,...,...,...,...
85751,77398,1449971,2020,CO,Colorado,West,Mountain,"Race, Ethnicity or Ancestry",Aggravated Assault,All
85752,77399,1449977,2020,CO,Colorado,West,Mountain,"Race, Ethnicity or Ancestry",Intimidation,All
85753,77400,1449982,2020,CO,Colorado,West,Mountain,Sexual Orientation,Simple Assault,All
85754,77401,1449982,2020,CO,Colorado,West,Mountain,Sexual Orientation,Theft From Building,All


In [13]:
# Delete a view - used during testing
with engine.connect() as con:
    con.execute(text('DROP VIEW population'))

## Database Testing

In [8]:
# Create database engine
engine = create_engine('sqlite:///us_hate_crimes_sqlite.db')

In [9]:
# Create inspector for testing
inspector = inspect(engine)

# Display tables and views in database
print(f'Tables: {inspector.get_table_names()}')
print(f'Views: {inspector.get_view_names()}')

# Display columns in bias table
display(inspector.get_columns('bias'))

Tables: ['bias', 'census_data', 'incident', 'incident_bias', 'incident_location', 'incident_offense', 'incident_victim_type', 'jurisdiction', 'location', 'offense', 'race', 'state', 'victim_type']
Views: ['incident_view', 'population_view', 'year_view']


[{'name': 'bias_id',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'bias',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'bias_category',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'primary_key': 0}]

In [10]:
# Reflect database and tables
metadata = MetaData()
metadata.reflect(engine)

# Define primary keys for all tables - this is needed to reflect them
Table(
    'bias',
    metadata,
    Column('bias_id', String, primary_key=True, nullable=False),
    autoload_with=engine,
    extend_existing=True
)

Table(
    'state',
    metadata,
    Column('state_abbr', String, primary_key=True, nullable=False),
    autoload_with=engine,
    extend_existing=True
)

Table(
    'population_view',
    metadata,
    Column('id', String, primary_key=True, nullable=False),
    autoload_with=engine
)

Table(
    'year_view',
    metadata,
    Column('id', String, primary_key=True, nullable=False),
    autoload_with=engine
)

Table(
    'incident_view',
    metadata,
    Column('id', String, primary_key=True, nullable=False),
    autoload_with=engine
)


Base = automap_base(metadata=metadata)
Base.prepare(autoload_with=engine)

In [11]:
B = Base.classes.bias
S = Base.classes.state
Y = Base.classes.year_view
I = Base.classes.incident_view
P = Base.classes.population_view

In [12]:
session = Session(engine)
results = session.query(S).with_entities(*[S.state, S.state_abbr]).all()
session.close()

for row in results:
    print(row)

('Alaska', 'AK')
('Alabama', 'AL')
('Arkansas', 'AR')
('Arizona', 'AZ')
('California', 'CA')
('Colorado', 'CO')
('Connecticut', 'CT')
('District of Columbia', 'DC')
('Delaware', 'DE')
('Florida', 'FL')
('Georgia', 'GA')
('Hawaii', 'HI')
('Iowa', 'IA')
('Idaho', 'ID')
('Illinois', 'IL')
('Indiana', 'IN')
('Kansas', 'KS')
('Kentucky', 'KY')
('Louisiana', 'LA')
('Massachusetts', 'MA')
('Maryland', 'MD')
('Maine', 'ME')
('Michigan', 'MI')
('Minnesota', 'MN')
('Missouri', 'MO')
('Mississippi', 'MS')
('Montana', 'MT')
('Nebraska', 'NB')
('North Carolina', 'NC')
('North Dakota', 'ND')
('New Hampshire', 'NH')
('New Jersey', 'NJ')
('New Mexico', 'NM')
('Nevada', 'NV')
('New York', 'NY')
('Ohio', 'OH')
('Oklahoma', 'OK')
('Oregon', 'OR')
('Pennsylvania', 'PA')
('Rhode Island', 'RI')
('South Carolina', 'SC')
('South Dakota', 'SD')
('Tennessee', 'TN')
('Texas', 'TX')
('Utah', 'UT')
('Virginia', 'VA')
('Vermont', 'VT')
('Washington', 'WA')
('Wisconsin', 'WI')
('West Virginia', 'WV')
('Wyoming', 'WY